In [4]:
from science_jubilee.bo.newcamera import Cam
from science_jubilee.labware.Labware import Well
from science_jubilee.Machine import Machine, get_root_dir
from science_jubilee.tools.Tool import Tool
from science_jubilee.decks.Deck import Deck
from science_jubilee.labware.Labware import Labware, Location, Well
from science_jubilee.tools.Pipette import Pipette
from science_jubilee.decks.Deck import Deck
import cv2
import numpy as np
import os
from jinja2 import Environment, FileSystemLoader, select_autoescape

In [8]:
deck = Deck("lab_automation_deck") # We are not using the deck, so operate any deck.json 
tiprack = deck.load_labware(labware_filename = 'opentrons_96_tiprack_20ul.json', slot = 1)   # Load an opentrons 20uL
tiprack.manual_offset([[269, 221.6],[170, 221.6],[170, 284.6]], save = True)   # Save through list of list. List of coordinates should be A1 A12 H12 subsequently. 
trash = deck.load_labware('agilent_1_reservoir_290ml.json', 0)
samples = deck.load_labware('nest_96_wellplate_200ul_flat.json', slot = 2)
samples.manual_offset([[127.4, 115.4],[27.4, 115.4], [27.4, 178.4]], save = True)
stocks = deck.load_labware('corning_6_wellplate_16.8ml.json', slot = 3)
stocks.manual_offset([[256.8, 127.4], [178.8, 127.4], [178.8, 165.4]], save = True)

New manual offset applied to opentrons_96_tiprack_20ul
Manual offset saved
New manual offset applied to nest_96_wellplate_200ul_flat
Manual offset saved
New manual offset applied to corning_6_wellplate_16.8ml_flat
Manual offset saved


In [32]:
samples.display_name
samples.slot

2

In [27]:
deck._get_slots().keys()

for i in deck._get_slots().keys():
    print(i)

0
1
2
3
4
5


In [5]:
camera = Cam(1, "Camera", '192.168.1.2', 'Sun92023', 'axo', 'C:/science-jubilee/axo/data', 50)

In [5]:
image_file_path = "C:/science-jubilee/src/science_jubilee/bo/image.jpg"  # Replace with your PNG file path

# Open the PNG file in binary mode and read its content
with open(image_file_path, 'rb') as f:
    image_bin = f.read()  # Read the image as binary data

# Decode the image using the provided function
#processor = ImageProcessor()
decoded_image = camera.decode_image(image_bin)

# Display the decoded image
cv2.imshow("Decoded Image", cv2.cvtColor(decoded_image, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV display
cv2.waitKey(0)
cv2.destroyAllWindows()
camera.get_rgb_avg(decoded_image)


[178.28820996331538, 118.11277980691187, 110.15860786716648]

## **Render json.file for camera** ##

In [6]:

# Known coordinates
A1_x, A1_y = 121.8, 73.6
A4_x, A4_y = 30.3, 73.6
D4_x, D4_y = 30.3, 134.8

# Generate coordinates for all wells
rows = ["A", "B", "C", "D"]
columns = [1, 2, 3, 4]

def camera_location(rows, columns, A1_x, A4_x, A4_y, D4_y):
    locations = []
    location_x = {}
    location_y = {}

    x_spacing = (A1_x - A4_x) / 3  # 3 intervals between 4 columns
    y_spacing = (D4_y - A4_y) / 3  # 3 intervals between 4 rows

    for row_index, row in enumerate(rows):
        for col_index, col in enumerate(columns):
            x = A1_x - (x_spacing * col_index)  # Subtract spacing for columns
            y = A1_y + (y_spacing * row_index)  # Add spacing for rows
            well_name = f"{row}{col}"

            # Append to locations and dictionaries
            locations.append(well_name)
            location_x[well_name] = round(x, 1)
            location_y[well_name] = round(y, 1)

    return locations, location_x, location_y

locations, location_x, location_y = camera_location(rows, columns, A1_x, A4_x, A4_y, D4_y)

# Prepare rendering
file_name = "camera_wellplate"

cam_config_path = os.path.join(get_root_dir(), "science_jubilee", "labware", "labware_definition", f"{file_name}.json")
template_dir = os.path.abspath("C:/science-jubilee/src/science_jubilee/labware/labware_definition")
env = Environment(loader=FileSystemLoader(template_dir))
template = env.get_template("camera_wellplate.json")

# Render the template
rendered_json = template.render(
    ordering=rows, locations=locations, location_x=location_x, location_y=location_y
)

# Write to a .json file
os.makedirs(os.path.dirname(cam_config_path), exist_ok=True)
with open(cam_config_path, "w") as file:
    file.write(rendered_json)

print(f"Labware definition saved to {cam_config_path}")
# Print the results
print("locations =", locations)
print("location_x =", location_x)
print("location_y =", location_y)

Labware definition saved to C:\science-jubilee\src\science_jubilee\labware\labware_definition\camera_wellplate.json
locations = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4', 'D1', 'D2', 'D3', 'D4']
location_x = {'A1': 121.8, 'A2': 91.3, 'A3': 60.8, 'A4': 30.3, 'B1': 121.8, 'B2': 91.3, 'B3': 60.8, 'B4': 30.3, 'C1': 121.8, 'C2': 91.3, 'C3': 60.8, 'C4': 30.3, 'D1': 121.8, 'D2': 91.3, 'D3': 60.8, 'D4': 30.3}
location_y = {'A1': 73.6, 'A2': 73.6, 'A3': 73.6, 'A4': 73.6, 'B1': 94.0, 'B2': 94.0, 'B3': 94.0, 'B4': 94.0, 'C1': 114.4, 'C2': 114.4, 'C3': 114.4, 'C4': 114.4, 'D1': 134.8, 'D2': 134.8, 'D3': 134.8, 'D4': 134.8}


In [15]:
# Known coordinates
A1_x, A1_y = 131.5, 70.6
A12_x, A12_y = 32.0, 70.6
H12_x, H12_y = 32.0, 133.4

# Generate coordinates for all wells
rows = ["A", "B", "C", "D", "E", "F", "G", "H"]
columns = list(range(1, 13))  # Columns 1 through 12

def camera_location(rows, columns, A1_x, A12_x, A12_y, H12_y):
    locations = []
    location_x = {}
    location_y = {}

    x_spacing = (A1_x - A12_x) / 11  # 11 intervals between 12 columns
    y_spacing = (H12_y - A12_y) / 7   # 7 intervals between 8 rows

    for row_index, row in enumerate(rows):
        for col_index, col in enumerate(columns):
            x = A1_x - (x_spacing * col_index)  # Subtract spacing for columns
            y = A1_y + (y_spacing * row_index)  # Add spacing for rows
            well_name = f"{row}{col}"

            # Append to locations and dictionaries
            locations.append(well_name)
            location_x[well_name] = round(x, 1)
            location_y[well_name] = round(y, 1)

    return locations, location_x, location_y

locations, location_x, location_y = camera_location(rows, columns, A1_x, A12_x, A12_y, H12_y)

# Prepare rendering
file_name = "cam_wellplate"

cam_config_path = os.path.join(get_root_dir(), "science_jubilee", "labware", "labware_definition", f"{file_name}.json")
template_dir = os.path.abspath("C:/science-jubilee/src/science_jubilee/labware/labware_definition")
env = Environment(loader=FileSystemLoader(template_dir))
template = env.get_template("camera_wellplate_template.json")

# Render the template
rendered_json = template.render(
    ordering=rows, locations=locations, location_x=location_x, location_y=location_y
)

# Write to a .json file
os.makedirs(os.path.dirname(cam_config_path), exist_ok=True)
with open(cam_config_path, "w") as file:
    file.write(rendered_json)

print(f"Labware definition saved to {cam_config_path}")
# Print the results
print("locations =", locations)
print("location_x =", location_x)
print("location_y =", location_y)

Labware definition saved to C:\science-jubilee\src\science_jubilee\labware\labware_definition\cam_wellplate.json
locations = ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12']
location_x = {'A1': 131.5, 'A2': 122.5, 'A3': 113.4, 'A4': 104.4, 'A5': 95.3, 'A6': 86.3, 'A7': 77.2, 'A8': 68.2, 'A9': 59.1, 'A10': 50.1, 'A11': 41.0, 'A12': 32.0, 'B1': 131.5, 'B2': 122.5, 'B3': 113.4, 'B4': 104.4, 'B5': 95.3, 'B6': 86.3, 'B7': 77.2, 'B8': 68.2, 'B9': 59.

## **Camera Testing from Jubilee Website** ##

In [2]:
camera = Cam(1, "Camera", '192.168.1.2', 'Sun92023', 'axo', 'C:/science-jubilee/axo/data', 50)

In [6]:
camera.take_picture(save_dir='C:/science-jubilee/axo/data')

TypeError: listdir: path should be string, bytes, os.PathLike or None, not Cam